In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

liste_saison = ["2023_2024", "2022_2023", "2021_2022", "2020_2021"]

saison = liste_saison[0]

event_import = pd.read_json(f"../../Data_file/Heatmap SB/{saison}.json")

In [73]:
[[]]*3

[[], [], []]

In [194]:
event = event_import[~event_import.location.isna()]

event.sort_values(by = ["match_id", "index"], inplace = True)

event.index = range(len(event))

centre = event[(event.pass_cross == 1) & (event.pass_type != "Corner")]
index = []
[index.extend(centre.index + i) for i in range (6)]

centre = pd.DataFrame({"attacking_team" : [i for i in centre.team for _ in range (6)],
                       "centre_id" : ["centre " + str(i) for i in range (1, len(centre) + 1) for _ in range (6)]},
                       index = [j + i for j in centre.index for i in range(6)])

df_centre = pd.concat([event.loc[centre.index], centre], axis = 1).set_index(["centre_id", "attacking_team"])

df_centre["But"] = 0

df_shot = df_centre[(df_centre.type == "Shot") & (df_centre.index.get_level_values(1) == df_centre.team)]

df_shot_player = df_shot.groupby(level = [0, 1]).player.transform(lambda x: ', '.join(x))
df_shot_player = df_shot_player.reset_index().drop_duplicates().set_index(["centre_id", "attacking_team"]).player

df_goal = df_centre[((df_centre.shot_outcome == "Goal") | (df_centre.type == "Own Goal For"))
                    & (df_centre.index.get_level_values(1) == df_centre.team)]

df_shot_player.name = "Tireur"
df_centre = pd.merge(df_centre, df_shot_player, left_index = True, right_index = True, how = "left")

df_centre.loc[df_goal.index, "But"] = 1
df_centre.loc[df_goal.index, "Buteur"] = df_goal.player

df_centre = df_centre.reset_index(level = 1)


mask_centre = df_centre.pass_cross == 1

df_loc_centre = pd.DataFrame(df_centre.loc[mask_centre, "location"].tolist(),
    index = df_centre.loc[mask_centre].index)
df_centre.loc[mask_centre, "x"] = df_loc_centre[0]
df_centre.loc[mask_centre, "y"] = df_loc_centre[1]

df_loc_centre_end = pd.DataFrame(df_centre.loc[mask_centre, "pass_end_location"].tolist(),
    index = df_centre.loc[mask_centre].index)
df_centre.loc[mask_centre, "x_end"] = df_loc_centre_end[0]
df_centre.loc[mask_centre, "y_end"] = df_loc_centre_end[1]


mask_shot = (df_centre.attacking_team == df_centre.team) & (df_centre.type == "Shot")

df_loc_shot = pd.DataFrame(df_centre.loc[mask_shot, "location"].tolist(),
    index = df_centre.loc[mask_shot].index)
df_centre.loc[mask_shot, "x"] = df_loc_shot[0]
df_centre.loc[mask_shot, "y"] = df_loc_shot[1]

df_loc_shot_end = pd.DataFrame(df_centre.loc[mask_shot, "shot_end_location"].tolist(),
    index = df_centre.loc[mask_shot].index)
df_centre.loc[mask_shot, "x_end"] = df_loc_shot_end[0]
df_centre.loc[mask_shot, "y_end"] = df_loc_shot_end[1]

df_centre = df_centre[["match_id", "attacking_team", "But", "Tireur", "Buteur", "player", "minute", "pass_body_part", "x", "y", "x_end", "y_end"]]
df_centre.rename()

,attacking_team,shot_type,shot_outcome,type,match_id,period,possession,location,team,pass_cross,...,player,shot_end_location,pass_body_part,But,Tireur,Buteur,x,y,x_end,y_end
centre_id,,,,,,,,,,,,,,,,,,,,,
centre 3,Rodez,None,None,Pass,3896583,1,13,"[99.1, 71.8]",Rodez,1.0,...,Ahmad Toure Ngouyamsa Nounchil,None,Right Foot,1,"Clément Dépres, Lorenzo Rajot",Lorenzo Rajot,99.1,71.8,107.0,48.3
centre 3,Rodez,None,None,Ball Receipt*,3896583,1,13,"[107.0, 48.3]",Rodez,NaN,...,Clément Dépres,None,None,1,"Clément Dépres, Lorenzo Rajot",Lorenzo Rajot,NaN,NaN,NaN,NaN
centre 3,Rodez,Open Play,Wayward,Shot,3896583,1,13,"[107.0, 48.3, 1.75]",Rodez,NaN,...,Clément Dépres,"[109.8, 36.7]",None,1,"Clément Dépres, Lorenzo Rajot",Lorenzo Rajot,107.0,48.3,109.8,36.7
centre 3,Rodez,None,None,Goal Keeper,3896583,1,13,"[1.4, 38.0]",AC Ajaccio,NaN,...,Mathieu Michel,None,None,1,"Clément Dépres, Lorenzo Rajot",Lorenzo Rajot,NaN,NaN,NaN,NaN
centre 3,Rodez,None,None,Ball Recovery,3896583,1,13,"[111.3, 32.9]",Rodez,NaN,...,Lorenzo Rajot,None,None,1,"Clément Dépres, Lorenzo Rajot",Lorenzo Rajot,NaN,NaN,NaN,NaN
centre 3,Rodez,Open Play,Goal,Shot,3896583,1,13,"[111.1, 34.2, 0.36]",Rodez,NaN,...,Lorenzo Rajot,"[120.0, 37.8, 1.3]",None,1,"Clément Dépres, Lorenzo Rajot",Lorenzo Rajot,111.1,34.2,120.0,37.8


In [148]:
df_centre = df_centre.reset_index(level = 1)
pd.concat([df_centre["match_id", "attacking_team", "But", "Tireur", "Buteur", "player", "minute"],
           pd.DataFrame(df_centre.location.tolist(), index = df_centre.index), pd.DataFrame(df_centre.pass_end_location.tolist(), index = df_centre.index)])

,attacking_team,shot_type,shot_outcome,type,match_id,period,possession,location,team,pass_cross,pass_type,index,pass_end_location,minute,player,shot_end_location,pass_body_part,But,Tireur,Buteur
centre_id,,,,,,,,,,,,,,,,,,,,
centre 1,AC Ajaccio,None,None,Pass,3896583,1,2,"[116.8, 67.8]",AC Ajaccio,1.0,None,19,"[111.7, 46.6]",0,Yacine Bammou,None,Right Foot,0,NaN,NaN
centre 1,AC Ajaccio,None,None,Clearance,3896583,1,2,"[8.4, 33.5]",Rodez,NaN,None,20,None,0,Eric Vandenabeele,None,None,0,NaN,NaN
centre 1,AC Ajaccio,None,None,Pass,3896583,1,3,"[93.7, 80.0]",AC Ajaccio,NaN,Throw-in,21,"[115.7, 66.3]",0,Mohamed Youssouf,None,None,0,NaN,NaN
centre 1,AC Ajaccio,None,None,Pressure,3896583,1,3,"[3.8, 17.5]",Rodez,NaN,None,22,None,0,Serge-Philippe Raux-Yao,None,None,0,NaN,NaN
centre 1,AC Ajaccio,None,None,Ball Receipt*,3896583,1,3,"[115.7, 66.3]",AC Ajaccio,NaN,None,23,None,0,Yacine Bammou,None,None,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
centre 9106,Rodez,None,None,Ball Receipt*,3937846,2,205,"[108.9, 42.6]",Rodez,NaN,None,3114,None,94,Taïryk Arconte,None,None,0,NaN,NaN
centre 9106,Rodez,None,None,Clearance,3937846,2,205,"[10.7, 40.7]",Saint-Étienne,NaN,None,3115,None,94,Dylan Batubinsika,None,None,0,NaN,NaN
centre 9106,Rodez,None,None,Ball Recovery,3937846,2,205,"[93.6, 33.5]",Rodez,NaN,None,3116,None,94,Wilitty Younoussa,None,None,0,NaN,NaN


In [18]:
df.loc[3:8]

,0
3,6
4,7
5,8
6,9
7,10
8,11


In [48]:
pd.concat([pd.Series(dico_event_test["Nb_but"], name = "Nombre but"), df_event_test], axis = 1).mean(axis = 0)

Nombre but        242.75
Ball Receipt*     244.25
Duel               24.75
Shot              246.50
Carry              23.00
Pressure           48.00
Error               3.25
Block               8.25
Ball Recovery      18.25
Goal Keeper         7.75
Clearance          11.50
Foul Committed      2.50
Pass                8.50
Miscontrol          0.75
Dribbled Past       0.25
Dribble             0.25
Foul Won            0.75
dtype: float64